In [1]:
library(tidyverse)
library(broom)
library(tableone)
library(ggplot2)
library(dplyr)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"package 'tableone' was built under R version 4.4.3"


In [ ]:
data <- read_csv("data_donnees_2021_ind_v2.csv")

head(data)
dim(data)

# data processing

In [ ]:
data <- data %>%
  mutate(
    post = ifelse(AGEGRP >= 17, 1, 0),  # AGEGRP 17+ corresponds to age 65+, so post = 1
    treat = case_when(
      IMMSTAT == 1 & HHInc_AT %in% 1:22 ~ 1,     # Canadian-born and income after tax < $90k -> eligible
      IMMSTAT == 1 & HHInc_AT >= 23 ~ 0,         # Canadian-born nd income after tax >= $90k -> ineligible
      IMMSTAT == 2 & AGEIMM <= 11 ~ 1,           # Immigrated before age 55 -> eligible
      IMMSTAT == 2 & AGEIMM > 11 ~ 0,            # Immigrated at or after age 55 -> ineligible 
      IMMSTAT == 3 ~ NA_real_,                   # Non-permanent residents -> drop
      AGEIMM %in% c(88, 99) ~ NA_real_           # Invalid age at immigration -> drop
    )
  )

In [ ]:
data_clean <- data %>%
  filter(
    AGEGRP %in% c(16, 17),     # Age 60–69
    IMMSTAT %in% c(1, 2),
    LFACT %in% 1:14,
    !is.na(LFACT),                   
    !is.na(Gender),
    !is.na(HDGREE),
    !is.na(EfDIMBM_2018),
    !is.na(post),
    !is.na(HHInc_AT), #try
    !is.na(treat)                   
  ) %>%
   mutate(
    labour_force = ifelse(LFACT %in% 1:10, 1, 0),      
    Gender = factor(Gender),
    HDGREE = factor(HDGREE),
    HHInc_AT = factor(HHInc_AT), #try
    EfDIMBM_2018 = factor(EfDIMBM_2018))

In [ ]:
table(data_clean$post, data_clean$treat)

# modeling

In [ ]:
model_dd_glm <- glm(
  labour_force ~ post * treat + Gender + HDGREE + EfDIMBM_2018 + HHInc_AT,
  data = data_clean,
  family = binomial
)

summary(model_dd_glm)

# visualization

## common trend

In [ ]:
pre_data <- data %>%
  filter(AGEGRP %in% c(13,14,15,16,17,18)) %>%
  mutate(
    labour_force = ifelse(LFACT %in% 1:10, 1, 0),
    treat = case_when(
      IMMSTAT == 1 & HHInc_AT %in% 1:22 ~ 1,      
      IMMSTAT == 1 & HHInc_AT >= 23 ~ 0,
      IMMSTAT == 2 & AGEIMM <= 11 ~ 1,
      IMMSTAT == 2 & AGEIMM > 11 ~ 0,
      IMMSTAT == 3 ~ NA_real_,
      AGEIMM %in% c(88, 99) ~ NA_real_
    )
  ) %>%
  filter(IMMSTAT %in% c(1,2), !is.na(treat), !is.na(labour_force)) %>%
  group_by(AGEGRP, treat) %>%
  summarise(lfp_rate = mean(labour_force), .groups = "drop")

# plot
library(ggplot2)

ggplot(pre_data, aes(x = AGEGRP, y = lfp_rate, color = factor(treat))) +
  geom_line(size = 1.2) +
  geom_point() +
  geom_vline(xintercept = 17, linetype = "dashed", color = "black") +
  labs(
    x = "Age Group (AGEGRP)",
    y = "Labour Force Participation Rate",
    color = "Treatment Group"
  ) +
  theme_minimal()


# post bar

In [ ]:
ggplot(data_clean, aes(x = factor(post), y = labour_force, fill = factor(treat))) +
  stat_summary(fun = mean, geom = "bar", position = "dodge") +
  labs(
    x = "Post (Age ≥ 65)",
    y = "Labour Force Participation Rate",
    fill = "Treatment Group"
  ) +
  theme_minimal()
